# Predicting if stock's value will go UP or DOWN with Long Short-Term Memory Network
The data is arranged in the following way:

X: stock's value for 50 consecutive days

y: 1 if stock's value on 51st day is higher than on 50th day, or 0 if it is not

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime

start = datetime.datetime(2006, 1, 1)
end = datetime.datetime(2018, 1, 1)
start_date_str = str(start.date())
end_date_str = str(end.date())

#how many consecutive prices are in X
sliding_window_size = 50

"""
stocks = ['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'XOM', 'GE',
          'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PFE',
          'PG', 'TRV', 'UTX', 'UNH', 'VZ', 'WMT', 'GOOGL', 'AMZN', 'AABA']

"""
stocks = ['AAPL']#, 'BA', 'CAT', 'CVX', 'CSCO', 'IBM']#, 'GOOGL', 'AMZN', 'AABA']

data_x = np.empty([sliding_window_size])
data_y = np.empty([1])
data_y_price = np.empty([1]) 

for stock in stocks:
    file_name = 'data/' + stock + '_' + start_date_str + '_to_' + end_date_str + '.csv'
    print(file_name)
    frame = pd.read_csv(file_name)
    
    #get only closing stock prices
    frame = frame['Close']
    
    frame =(frame.values).reshape(-1,1)
    data = (normalize(frame, axis=0)).squeeze()
    print("Days: ", data.size)
    
    
    for i in range(data.size - sliding_window_size - 1):
        data_sample_x = data[i:i+sliding_window_size]

        #EITHER EXACT PRICE OR INCREASE/DECREASE FORMAT
        data_sample_y_price = data[i+sliding_window_size]
        data_sample_y = 1 if data[i+sliding_window_size] > data[i+sliding_window_size - 1] else 0
        data_x = np.vstack([data_x , data_sample_x])
        data_y = np.vstack([data_y , data_sample_y])
        data_y_price = np.vstack([data_y_price , data_sample_y_price])
#remove 1st element, which was created by np.empty
data_x = data_x[1:]
data_y = data_y[1:]
data_y_price = data_y_price[1:]

print("X: ", data_x.shape)
print("y: ", data_y.shape)

data/AAPL_2006-01-01_to_2018-01-01.csv
3019
(2968, 50)
(2968, 1)
(2968, 1)


### Split data into training, validation, and test datasets

In [2]:

number_of_samples = data_x.shape[0]

validation_first_index = int(number_of_samples * 0.7)
testing_first_index = int(number_of_samples * (0.7 + 0.15))

data_x_train = data_x[:validation_first_index]
data_y_train = data_y[:validation_first_index]

data_x_val = data_x[validation_first_index:testing_first_index]
data_y_val = data_y[validation_first_index:testing_first_index]

data_x_test = data_x[testing_first_index:]
data_y_test = data_y[testing_first_index:]

### Transform data to PyTorch tensors

In [3]:

X_train_tensors = Variable(torch.Tensor(data_x_train))
X_val_tensors = Variable(torch.Tensor(data_x_val))
X_test_tensors = Variable(torch.Tensor(data_x_test))

y_train_tensors = Variable(torch.Tensor(data_y_train))
y_val_tensors = Variable(torch.Tensor(data_y_val)) 
y_test_tensors = Variable(torch.Tensor(data_y_test)) 

print("Training X: ", X_train_tensors.shape)
print("Training y: ", y_train_tensors.shape)
print("Validation X: ", X_val_tensors.shape)
print("Validation y: ", y_val_tensors.shape)
print("Testing X: ", X_test_tensors.shape)
print("Testing y: ", y_test_tensors.shape)

torch.Size([2077, 50])
torch.Size([2077, 1])
torch.Size([445, 50])
torch.Size([445, 1])
torch.Size([446, 50])
torch.Size([446, 1])


### Change dimensions to (rows, timestamps, features)

In [4]:
X_train_tensors_final = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))

X_val_tensors_final = torch.reshape(X_val_tensors,   (X_val_tensors.shape[0], 1, X_val_tensors.shape[1]))

X_test_tensors_final = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1])) 

print("Training Shape:", X_train_tensors_final.shape, y_train_tensors.shape)
print("Validation Shape:", X_val_tensors_final.shape, y_val_tensors.shape)
print("Testing Shape:", X_test_tensors_final.shape, y_test_tensors.shape) 

Training Shape: torch.Size([2077, 1, 50]) torch.Size([2077, 1])
Validation Shape: torch.Size([445, 1, 50]) torch.Size([445, 1])
Testing Shape: torch.Size([446, 1, 50]) torch.Size([446, 1])


### Long Short-Term Memory Network with 2 Linear Layers

In [5]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers 
        self.input_size = input_size 
        self.hidden_size = hidden_size
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 64) #fully connected 1
        self.fc_2 =  nn.Linear(128, 64) #fully connected 2
        self.fc = nn.Linear(64, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.rand(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.rand(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        #print(hn.shape)
        #is layers > 1, get only last element
        hn = hn[-1]
        #print(hn.shape)
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        #print(hn.shape)
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        #out = self.fc_2(out) #2nd Dense
        #out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

### Training

In [6]:
num_epochs = 1000
learning_rate = 0.001 #0.001 lr

input_size = 50 #number of features
hidden_size = 10 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 1 #number of output classes 

lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1])

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate) 

lowest_loss = 999999
for epoch in range(num_epochs):
    output = lstm1.forward(X_train_tensors_final) #forward pass
    optimizer.zero_grad()
 
    loss = criterion(output, y_train_tensors)
 
    loss.backward() #calculates the loss of the loss function
 
    optimizer.step()
    
    #validation and early stopping
    lstm1.eval()
    output = lstm1(X_val_tensors_final)
    val_loss = criterion(output, y_val_tensors)
    if val_loss.item() < lowest_loss:
        lowest_loss = val_loss.item()
        torch.save(lstm1.state_dict(), "checkpoint.pt")
    
    if epoch % 100 == 0:
        print("Epoch: %d, training loss: %1.5f, validation loss: %1.5f" % (epoch, loss, val_loss))

    

# load the last checkpoint with the best model
lstm1.load_state_dict(torch.load('checkpoint.pt'))


Epoch: 0, training loss: 0.49568, validation loss: 0.47242
Epoch: 100, training loss: 0.24987, validation loss: 0.25199
Epoch: 200, training loss: 0.24991, validation loss: 0.25127
Epoch: 300, training loss: 0.24985, validation loss: 0.25023
Epoch: 400, training loss: 0.24970, validation loss: 0.25035
Epoch: 500, training loss: 0.24975, validation loss: 0.25006
Epoch: 600, training loss: 0.24938, validation loss: 0.25075
Epoch: 700, training loss: 0.24978, validation loss: 0.24996
Epoch: 800, training loss: 0.24981, validation loss: 0.25027
Epoch: 900, training loss: 0.24965, validation loss: 0.25008


<All keys matched successfully>

### Testing

In [7]:
with torch.no_grad():
    output = torch.round(lstm1.forward(X_test_tensors_final))
    acc = torch.eq(output, y_test_tensors)
    print("correctness: %1.5f" % ((sum(acc)/ y_test_tensors.shape[0]).item()))
    print("loss: %1.5f" % (criterion(output, y_test_tensors)))


correctness: 0.55830
loss: 0.44170


In [8]:

#plt.plot(output, label='prediction')
#plt.plot(y_test_tensors, label='real')
#plt.legend()
#plt.show()

In [9]:
#generative mode
"""
X_new = X_test_tensors_final[0:1]
y_test_tensors
print(X_new.shape, X_test_tensors_final.shape, y_test_tensors.shape)
output = []
with torch.no_grad():
    for i in range(100):
        
        out = lstm1.forward(X_new)
        output.append(out)
        
        X_new = X_new[:, :, 1:50]

        X_new = torch.cat((X_new, out.unsqueeze(1)), dim=2)


good = 0
bad = 0
last_pred = 0
last_true = 0
for pred_y, real_y in zip(output, y_test_tensors):
    if pred_y > last_pred and real_y > last_true:
        good+=1
    elif pred_y > last_pred and real_y < last_true:
        bad+=1
    elif pred_y < last_pred and real_y < last_true:
        good+=1
    elif pred_y < last_pred and real_y > last_true:
        bad+=1
    last_pred = pred_y
    last_true = real_y
print(good, bad, output.shape, y_test_tensors)
"""

'\nX_new = X_test_tensors_final[0:1]\ny_test_tensors\nprint(X_new.shape, X_test_tensors_final.shape, y_test_tensors.shape)\noutput = []\nwith torch.no_grad():\n    for i in range(100):\n        \n        out = lstm1.forward(X_new)\n        output.append(out)\n        \n        X_new = X_new[:, :, 1:50]\n\n        X_new = torch.cat((X_new, out.unsqueeze(1)), dim=2)\n\n\ngood = 0\nbad = 0\nlast_pred = 0\nlast_true = 0\nfor pred_y, real_y in zip(output, y_test_tensors):\n    if pred_y > last_pred and real_y > last_true:\n        good+=1\n    elif pred_y > last_pred and real_y < last_true:\n        bad+=1\n    elif pred_y < last_pred and real_y < last_true:\n        good+=1\n    elif pred_y < last_pred and real_y > last_true:\n        bad+=1\n    last_pred = pred_y\n    last_true = real_y\nprint(good, bad, output.shape, y_test_tensors)\n'

### Experiment
Let's invest using our model and see if me make a profit!

In [10]:
money = 1000

#get prices matching the test dataset
data_y_price_test = data_y_price[testing_first_index:]
with torch.no_grad():
    output = torch.round(lstm1.forward(X_test_tensors_final))

for i in range(1, data_y_price_test.shape[0]):
 
    #if the model predicts that the price will rise then hold it
    if output[i].item()==1:
        money = money * data_y_price_test[i].item() / data_y_price_test[i-1].item()
    #if the model predicts that the price will decrease then sell it and buy it the next day
    else:
        money = money * data_y_price_test[i-1].item() / data_y_price_test[i].item()

print("Money after investing: %1.2f" % money)

Money after investing: 1499.10


In [11]:
#print(len([i for i in output if i ==1]), len([i for i in output if i ==0]))